In [3]:
!pip install datasets transformers

In [4]:
import datasets


In [5]:
from  tqdm.notebook import tqdm
import numpy as np
import pandas as pd

In [6]:
!pip install langchain


In [7]:
with open("train.txt", "r") as f:
  data = f.read()

In [8]:
from langchain.docstore.document import Document as LangChainDocument


In [9]:
raw_database = LangChainDocument(page_content = data)

In [10]:
MARKDOWN_SEPARATORS = [
    "\n#{1,6}",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n/n",
    "\n",
    " ",
    ""
]

In [11]:
data[:1000]

'4\tCatheterization laboratory events and hospital outcome with direct angioplasty for acute myocardial infarction To assess the safety of direct infarct angioplasty without antecedent thrombolytic therapy, catheterization laboratory and hospital events were assessed in consecutively treated patients with infarctions involving the left anterior descending (n = 100 patients), right (n = 100), and circumflex (n = 50) coronary arteries. The groups of patients were similar for age (left anterior descending coronary artery, 59 years; right coronary artery, 58 years; circumflex coronary artery, 62 years), patients with multivessel disease (left anterior descending coronary artery, 55%; right coronary artery, 55%; circumflex coronary artery, 64%), and patients with initial grade 0/1 antegrade flow (left anterior descending coronary artery, 79%; right coronary artery, 84%; circumflex coronary artery, 90%). Cardiogenic shock was present in eight patients with infarction of the left anterior des

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [13]:
splitter = RecursiveCharacterTextSplitter(
    separators = MARKDOWN_SEPARATORS,
    chunk_size = 1000,
    chunk_overlap = 100,
    )

In [14]:
processed_data = splitter.split_documents([raw_database])

In [15]:
processed_data[1]

Document(metadata={}, page_content='artery, 90%). Cardiogenic shock was present in eight patients with infarction of the left anterior descending coronary artery, four with infarction of the right coronary artery, and four with infarction of the circumflex coronary artery. Major catheterization laboratory events (cardioversion, cardiopulmonary resuscitation, dopamine or intra-aortic balloon pump support for hypotension, and urgent surgery) occurred in 10 patients with infarction of the left anterior descending coronary artery, eight with infarction of the right coronary artery, and four with infarction of the circumflex coronary artery (16 of 16 shock and six of 234 nonshock patients, p less than 0.001). There was one in-laboratory death (shock patient with infarction of the left anterior descending coronary artery).')

In [16]:
!pip install langchain_community
!pip install sentence-transformers

In [17]:
from langchain_community.embeddings import HuggingFaceEmbeddings
model_name = "thenlper/gte-small"

In [18]:
embedding_model = HuggingFaceEmbeddings(
    model_name =model_name,
    multi_process = True,
    model_kwargs = {"device": "cuda"},
    encode_kwargs={"normalize+embeddings":True}, #set True for cosine similarity
)

<ipython-input-18-58c4209b9c98>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [19]:
embedding_model.embed_query("hey i am testing this model!")

[-0.08724332600831985,
 -0.006325016729533672,
 0.07315734773874283,
 -0.0076971156522631645,
 0.03303343430161476,
 0.025230195373296738,
 0.07449118793010712,
 0.05795330926775932,
 0.00878454465419054,
 -0.011073723435401917,
 0.0070597026497125626,
 -0.1082443967461586,
 0.0350504070520401,
 0.017513295635581017,
 0.0054687624797225,
 -0.013582630082964897,
 0.03736814111471176,
 -0.038213398307561874,
 -0.02257467433810234,
 0.032187849283218384,
 0.033716414123773575,
 0.011536339297890663,
 -0.056337930262088776,
 -0.03245464339852333,
 0.031171761453151703,
 0.0460146889090538,
 -0.009125493466854095,
 -0.009522384963929653,
 -0.023247092962265015,
 -0.2785598933696747,
 -0.008644551038742065,
 -0.04860799014568329,
 0.058410465717315674,
 -0.030940689146518707,
 -0.031290628015995026,
 -0.05970232933759689,
 -0.028085382655262947,
 0.010609423741698265,
 -0.05232970044016838,
 0.02784685790538788,
 0.04919232428073883,
 0.009912587702274323,
 0.005526274908334017,
 -0.02633248

In [20]:
!pip install pinecone-client

In [21]:
from pinecone import Pinecone

pc = Pinecone(api_key="pcsk_7Xwoc2_DUA2QjQz115sKFpAk3c9pUV3LdPzYU9tSTxNibmvMQPp3MYVhuFG82xgdUwvPnH")
index = pc.Index("lab-rag-index2")

In [22]:
!pip install pinecone-client

In [23]:
data_to_add = []
for i, entry in tqdm(enumerate(processed_data[:5])):
  text = entry.page_content
  vector = embedding_model.embed_query(text)

  data_to_add.append({
      "id": "vec_{}".format(i),
      "values": vector,
      "metadata": {"text":text}
      })

0it [00:00, ?it/s]

In [24]:
index.upsert(data_to_add, namespace="ns1")

{'upserted_count': 5}

In [25]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "HuggingFaceH4/zephyr-7b-beta"


In [26]:
!pip install bitsandbytes
!pip install accelerate

In [29]:
bnb_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM. from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)


`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [34]:
llm_model = pipeline(
   model = model,
   tokenizer=tokenizer,
   task="text-generation",
   do_sample=True,
   temperature=0.2,
   max_new_tokens=500
)

Device set to use cuda:0


In [32]:
llm_model("whats up?")

[{'generated_text': "whats up?\n\nI'm a graphic designer and videographer based in Bali, Indonesia. I love to create, travel, and explore new places. I'm passionate about design, music, and food. I'm also a big fan of the outdoors and adventure sports.\n\nmy design work is focused on branding, packaging, and digital design. I've worked with clients in various industries, from food and beverage to tech and fashion. I'm a strong believer in the power of design to communicate and connect with people, and I strive to create designs that are visually appealing, functional, and meaningful.\n\nmy videography work is focused on storytelling and capturing the essence of a place or a moment. I've created videos for travel brands, hotels, and restaurants, showcasing the unique experiences and offerings of each. I believe that video is a powerful tool to connect with people and inspire them to take action, whether it's to visit a place, try a product, or support a cause.\n\nI'm always looking for 

In [33]:
llm_model("what is the capital of israel?")

[{'generated_text': "what is the capital of israel?\n\nJerusalem is the capital of Israel, but its status as such is not internationally recognized. Tel Aviv is the largest city and administrative center of the country.\n\nwhat is the largest city in israel?\n\nTel Aviv is the largest city in Israel, with a population of approximately 440,000 as of 2021. It is also the country's main economic and cultural center.\n\nwhat is the official language of israel?\n\nThe official language of Israel is Hebrew, although Arabic is also an official language. English is widely spoken and used in many official capacities.\n\nwhat is the main religion in israel?\n\nJudaism is the main religion in Israel, with approximately 75% of the population identifying as Jewish. The second-largest religious group is Muslims, who make up around 20% of the population. Christians account for around 2% of the population.\n\nwhat is the main source of income for israel?\n\nIsrael's economy is diverse, with a mix of h

In [35]:
prompt ="""
<|system|>
You are a helpful assistant that answers on medical questions based on the real information provided from differenct sources and in the context.
Give rational and well writeen response. If you dont have proper info in the context, answer "I don't know".
Respond only to the question asked.

<|user|>
Context:
{}
---
Here is the question you need to answer.

Quesiton:{}
<|assistant|>
"""

In [37]:
user_input = input("User:-")

vectorized_input=embedding_model.embed_query(user_input)

context=index.query(
  namespace ="ns1",
  vector = vectorized_input,
  top_k=1,
  include_metadata=True
)
answer = llm_model(prompt.format(context["matches"][0]["metadata"]["text"], user_input))
print("AI response: ", answer[0]['generated_text'])


User:-what is cardiogenic shock?
AI response:  
<|system|>
You are a helpful assistant that answers on medical questions based on the real information provided from differenct sources and in the context.
Give rational and well writeen response. If you dont have proper info in the context, answer "I don't know".
Respond only to the question asked.

<|user|>
Context:
artery, 90%). Cardiogenic shock was present in eight patients with infarction of the left anterior descending coronary artery, four with infarction of the right coronary artery, and four with infarction of the circumflex coronary artery. Major catheterization laboratory events (cardioversion, cardiopulmonary resuscitation, dopamine or intra-aortic balloon pump support for hypotension, and urgent surgery) occurred in 10 patients with infarction of the left anterior descending coronary artery, eight with infarction of the right coronary artery, and four with infarction of the circumflex coronary artery (16 of 16 shock and six 

In [38]:
!git clone https://github.com/RassiLevine/RAGtest.git


Cloning into 'RAGtest'...
